In [ ]:
import pandas as pd
import seaborn
import numpy as np

### Importing Data

In [ ]:
df = pd.read_csv("data/ytdf3.csv")
df.head(5)

In [ ]:
df.info()

### Cleaning data

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize